In [4]:
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras import losses
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [3]:
y_train = np.zeros((len(Y_train),10))
y_test = np.zeros((len(Y_test),10))
for index in range(len(Y_train)):
    y_train[index][Y_train[index]] = 1
for index in range(len(Y_test)):
    y_test[index][Y_test[index]] = 1
Y_train = y_train
Y_test = y_test

In [4]:
X_train = X_train/255
X_test = X_test/255
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000, 10)
(10000, 10)


In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000, 10)
(10000, 10)


In [6]:
def mnistModel(input_shape):
    X_Input = Input(input_shape)

    X = Conv2D(30, (5, 5), padding = "same", name = 'conv1')(X_Input)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool_1')(X)

    X = Conv2D(50, (5, 5), padding = "same", name = 'conv2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool_2')(X)

    X = Flatten()(X)
    X = Dense(500, activation='relu', name='fc1')(X)
    X = Dense(10, activation='softmax', name='fc2')(X)

    model = Model(inputs = X_Input, outputs = X, name='decaptcha')
    return model

In [7]:
model = mnistModel(X_train.shape[1:])

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
model.fit(x = X_train, y = Y_train, batch_size=32, epochs=2, verbose=1)

Epoch 1/2
60000/60000 [==============================] - 198s 3ms/step - loss: 0.1021 - accuracy: 0.9683
Epoch 2/2
60000/60000 [==============================] - 183s 3ms/step - loss: 0.0368 - accuracy: 0.9888


In [10]:
pred = model.evaluate(x = X_test,y = Y_test)
print()
print ("Loss = " + str(pred[0]))
print ("Test Accuracy = " + str(pred[1]))

10000/10000 [==============================] - 11s 1ms/step

Loss = 0.030876718425017317
Test Accuracy = 0.9897000193595886


In [12]:
model.save('mnist.model')

In [15]:
new_model = keras.models.load_model('mnist.model')
img = cv2.imread('test.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28,28), interpolation = cv2.INTER_AREA)
img = img[np.newaxis,...,np.newaxis]
predictions = new_model.predict([img])
predictions

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [16]:
np.argmax(predictions, axis = 1)

array([8])